# Dataset Optimization - Rings

To reproduce this experiment, first delete or remove the database rings.db that is stored in the results/optimization folder. Otherwise, the TPE optimization will use the 100 trials stored in the existing rings.db and continue with the 101st trial up to the 200th trial. 

In [12]:
import sys  
sys.path.insert(0, '../..')

In [ ]:
%load_ext autoreload
%autoreload 2

In [14]:
import optuna
from optuna.samplers import TPESampler
import numpy as np
import pandas as pd

from sklearn.datasets import make_blobs, make_moons, make_circles
from sklearn.cluster import KMeans, DBSCAN, SpectralClustering, MeanShift, estimate_bandwidth
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import adjusted_mutual_info_score as AMI
from rock import ROCK

import logging
import sys

In [15]:
def objective(trial):
    seed = 0

    rock_results = []
    other_results = []

    n_samples = trial.suggest_int('n_samples', 1, 16) * 100
    jitter = trial.suggest_int('jitter', 1, 20) * 0.01

    datasets = []

    for s in range(seed, seed+10):
        datasets.append(make_circles(n_samples=n_samples,
                                     noise=jitter, 
                                     random_state=s))
        
    step = 0
    for dataset in datasets: 
        run = {}
        run['step'] = step
        run['n_samples'] = n_samples
        run['jitter'] = jitter
        scaler = StandardScaler()
        data = scaler.fit_transform(dataset[0])
        kmeans = KMeans(n_clusters=2, random_state=seed).fit(data).labels_
        run['k'] = 2
        eps, min_pts = (0.2, 4)
        dbscan = DBSCAN(eps=eps, min_samples=min_pts).fit(data).labels_
        run['eps'] = eps
        run['min_pts'] = min_pts
        spectral = SpectralClustering(n_clusters=2, random_state=seed).fit(data).labels_
        bandwidth = estimate_bandwidth(data)
        run['bandwidth'] = bandwidth
        mean_shift = MeanShift(bandwidth=bandwidth).fit(data).labels_

        rock = ROCK(tmax=15).fit(data).labels_

        gt = dataset[1]
        rock_results.append(AMI(rock, gt))
        other_results.append(np.max([AMI(kmeans, gt), AMI(dbscan, gt), AMI(spectral, gt), AMI(mean_shift, gt)]))

        run['ROCK'] = AMI(rock, gt)
        run['K_MEANS'] = AMI(kmeans, gt)
        run['DBSCAN'] = AMI(dbscan, gt)
        run['SPECTRAL'] = AMI(spectral, gt)
        run['MEAN_SHIFT'] = AMI(mean_shift, gt)

        step += 1

        experiment.append(run)

    return np.mean(rock_results) - np.mean(other_results)

In [16]:
experiment = []

In [17]:
study_name = 'rings'

In [18]:
storage_name = f'sqlite:///../../results/optimization/{study_name}.db'
sampler = TPESampler(seed=42)
study = optuna.create_study(study_name=study_name, storage=storage_name, sampler=sampler, load_if_exists=True,  direction="maximize")

[I 2021-06-01 16:17:45,623] A new study created in RDB with name: rings


In [19]:
study.optimize(objective, n_trials=100)

[I 2021-06-01 16:18:36,069] Trial 0 finished with value: -0.02142656584239766 and parameters: {'n_samples': 6, 'jitter': 20}. Best is trial 0 with value: -0.02142656584239766.
[I 2021-06-01 16:19:46,897] Trial 1 finished with value: 0.0019477457941735627 and parameters: {'n_samples': 12, 'jitter': 12}. Best is trial 1 with value: 0.0019477457941735627.
[I 2021-06-01 16:19:54,637] Trial 2 finished with value: -0.03451547094700948 and parameters: {'n_samples': 3, 'jitter': 4}. Best is trial 1 with value: 0.0019477457941735627.
[I 2021-06-01 16:19:58,094] Trial 3 finished with value: -0.006095386668716002 and parameters: {'n_samples': 1, 'jitter': 18}. Best is trial 1 with value: 0.0019477457941735627.
[I 2021-06-01 16:20:55,043] Trial 4 finished with value: -0.010851512210408872 and parameters: {'n_samples': 10, 'jitter': 15}. Best is trial 1 with value: 0.0019477457941735627.
[I 2021-06-01 16:20:58,590] Trial 5 finished with value: -0.007955213696754998 and parameters: {'n_samples': 1, 

[I 2021-06-01 16:45:45,192] Trial 44 finished with value: 0.04893359520426764 and parameters: {'n_samples': 11, 'jitter': 3}. Best is trial 18 with value: 0.17597702663426662.
[I 2021-06-01 16:46:09,521] Trial 45 finished with value: -0.8382428061354813 and parameters: {'n_samples': 9, 'jitter': 1}. Best is trial 18 with value: 0.17597702663426662.
[I 2021-06-01 16:46:51,500] Trial 46 finished with value: 0.014908427039648557 and parameters: {'n_samples': 10, 'jitter': 5}. Best is trial 18 with value: 0.17597702663426662.
[I 2021-06-01 16:48:28,118] Trial 47 finished with value: -0.011764404984306665 and parameters: {'n_samples': 13, 'jitter': 19}. Best is trial 18 with value: 0.17597702663426662.
[I 2021-06-01 16:48:47,205] Trial 48 finished with value: 0.011689196725315465 and parameters: {'n_samples': 6, 'jitter': 4}. Best is trial 18 with value: 0.17597702663426662.
[I 2021-06-01 16:49:16,268] Trial 49 finished with value: 0.0036874326375312474 and parameters: {'n_samples': 7, 'jit

[I 2021-06-01 17:20:18,042] Trial 88 finished with value: 0.0076502401050678554 and parameters: {'n_samples': 15, 'jitter': 5}. Best is trial 86 with value: 0.1789493922434452.
[I 2021-06-01 17:22:05,478] Trial 89 finished with value: 0.0018143812211078294 and parameters: {'n_samples': 16, 'jitter': 11}. Best is trial 86 with value: 0.1789493922434452.
[I 2021-06-01 17:23:09,213] Trial 90 finished with value: 0.008229325404744595 and parameters: {'n_samples': 15, 'jitter': 4}. Best is trial 86 with value: 0.1789493922434452.
[I 2021-06-01 17:23:59,835] Trial 91 finished with value: 0.1776092548696203 and parameters: {'n_samples': 14, 'jitter': 2}. Best is trial 86 with value: 0.1789493922434452.
[I 2021-06-01 17:24:49,725] Trial 92 finished with value: 0.1776092548696203 and parameters: {'n_samples': 14, 'jitter': 2}. Best is trial 86 with value: 0.1789493922434452.
[I 2021-06-01 17:25:31,512] Trial 93 finished with value: -0.8402494518463727 and parameters: {'n_samples': 14, 'jitter':

In [20]:
pd.DataFrame(experiment).to_csv('../../results/optimization/rings.csv')